In [55]:
import os
import re
import sys
import copy

ROOT = os.path.join(os.path.dirname(os.getcwd()), 'backend')
if ROOT not in sys.path:
    sys.path.append(ROOT)
    
from moz_sql_parser import parse as ps
sys.setrecursionlimit(1000000)
import json
import os
import pandas as pd
from termcolor import colored
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import AgglomerativeClustering
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
import plotly.express as px

In [56]:
from app.dataService import globalVariable as GV
# from app.dataService.dataService import DataService
print(f"GV.SPIDER_FOLDER: {GV.SPIDER_FOLDER}")
from app.dataService.utils.processSQL import process_sql
from app.dataService.utils.processSQL import decode_sql
from app.dataService import sql2sql

db_schema, db_names, tables = process_sql.get_schemas_from_json(os.path.join(GV.SPIDER_FOLDER, "tables.json"))
# dataService = DataService("spider")

GV.SPIDER_FOLDER: /data2/xingbo/chi2022/seqNLI/backend/app/dataService/../data/dataset/spider


In [57]:
test_data_type = "dev"
with open(os.path.join(GV.SPIDER_FOLDER, f"{test_data_type}.json"), "r") as f:
    test_data = pd.DataFrame(json.load(f))

In [58]:
# employee_hire_evaluation
topic_to_search = "employee_hire_evaluation"
# test_data[test_data.db_id==topic_to_search]
table_2_search_cols =  [tables[topic_to_search]["table_names"][col[0]] + ": "+col[1] for col in tables[topic_to_search]["column_names"] if col[1]!="*"]
table_2_search_cols += [table_name + ": *" for table_name in tables[topic_to_search]["table_names"]]
table_2_search_cols


['employee: employee id',
 'employee: name',
 'employee: age',
 'employee: city',
 'shop: shop id',
 'shop: name',
 'shop: location',
 'shop: district',
 'shop: number products',
 'shop: manager name',
 'hiring: shop id',
 'hiring: employee id',
 'hiring: start from',
 'hiring: is full time',
 'evaluation: employee id',
 'evaluation: year awarded',
 'evaluation: bonus',
 'employee: *',
 'shop: *',
 'hiring: *',
 'evaluation: *']

In [59]:
dataset_type = "train_spider"
with open(os.path.join(GV.SPIDER_FOLDER, f"{dataset_type}.json"), "r") as f:
    spider_data = json.load(f)
spider_df = pd.DataFrame(spider_data)
spider_df.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."


In [60]:
topic_sim_th = 0.4

def cal_cosine_sim(sen0, sen1):
    embedd0 = model.encode(sen0, convert_to_tensor=True)
    embedd1 = model.encode(sen1, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embedd0, embedd1).cpu().numpy()
    return cosine_scores
    
def search_sim_dbs(topic, db_names):
    sim_scores = cal_cosine_sim(topic, db_names)
    return sim_scores

new_db_names = [re.sub(r'[0-9]+', '', n.replace("_", " ")).strip() for n in db_names]

sim_scores = search_sim_dbs(topic_to_search, new_db_names)[0]

df = pd.DataFrame({
    "sim_scores": list(sim_scores),
    "old_db_names": db_names,
    "db_names": new_db_names
})

df = df.sort_values("sim_scores", ascending=False).reset_index(drop=True)
if df.iloc[0]["sim_scores"]<topic_sim_th:
    df = df.iloc[:3]
else:
    df = df[df.sim_scores>topic_sim_th].reset_index(drop=True)
df.head()

,sim_scores,old_db_names,db_names
0,0.909449,employee_hire_evaluation,employee hire evaluation
1,0.574030,products_for_hire,products for hire
2,0.560887,company_employee,company employee
3,0.472382,department_management,department management
4,0.417009,hr_1,hr


In [61]:
spider_df = spider_df[spider_df["db_id"].isin(df.old_db_names.values)].reset_index(drop=True)
spider_df.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."


In [62]:

def col_unit_2_string(col_unit):
    if col_unit is not None:
        agg_id, col, isDistinct = col_unit
        if agg_id != "none":
            return f"{isDistinct} {agg_id} ({col})".strip() #(isDistinct + " " + agg_id + " (" + col + ")").strip()
        else:
            return f"{isDistinct} {col}".strip()
    else:
        return ""
def extract_select_ent(select):
    cols_ents = []
    global_distinct = select[0]
    for s in select[1]:
        agg_id = s[0]
        val_unit = s[1]
        # 'select': (isDistinct(bool), [(agg_id, val_unit), (agg_id, val_unit), ...])
        # val_unit: (unit_op, col_unit1, col_unit2)
        # col_unit: (agg_id, col_id, isDistinct(bool))
        unit_op, col_unit1, col_unit2 = val_unit
        agg_id1, col1, isDistinct1 = col_unit1
        col1_string = col_unit_2_string(col_unit1)
        
        if col_unit2 is not None:
            col2_string = col_unit_2_string(col_unit2)
            col1_string =  " ".join([col1_string, unit_op, col2_string])
        
        if agg_id != "none":
            col1_string = agg_id + " (" + col1_string + ")"
        cols_ents.append(col1_string)
    return cols_ents

def extract_select_names(select):
    select_names = []
    for s in select[1]:
        agg_id = s[0]
        val_unit = s[1]
        unit_op, col_unit1, col_unit2 = val_unit
        col_string = col_unit1[1]
        if col_unit2 is not None:
            col_string += ", " + col_unit2[1]
        select_names.append(col_string)
    return select_names

record_count = 0

sim_query_list = []

for rowid, row in spider_df.iterrows():
    sql = row["query"]
    db_name = row["db_id"]
    table = tables[db_name]
    schema = process_sql.Schema(db_schema[db_name], table)
    sql_label = process_sql.get_sql(schema, sql)
    select_decoded = decode_sql.decode_select(sql_label, table)
    select_name =  extract_select_names(select_decoded)
    select_ents = extract_select_ent(select_decoded)
    record_count += 1
    row_sim = np.max(cal_cosine_sim(table_2_search_cols, select_name), axis=1)
    sim_query_list.append(row_sim)

sim_query_list = np.array(sim_query_list)
        
fig = px.imshow(sim_query_list)
fig.show()

In [79]:
sim_query_list_binary = pd.DataFrame(np.where(sim_query_list > topic_sim_th, 1, 0), columns = table_2_search_cols)
fig = px.imshow(sim_query_list_binary)
fig.show()

In [89]:
sim_query_list_binary

,employee: employee id,employee: name,employee: age,employee: city,shop: shop id,shop: name,shop: location,shop: district,shop: number products,shop: manager name,...,hiring: employee id,hiring: start from,hiring: is full time,evaluation: employee id,evaluation: year awarded,evaluation: bonus,employee: *,shop: *,hiring: *,evaluation: *
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,1
1,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
250,0,0,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,1,1,1,0
251,0,0,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,1,1,1,0
252,0,1,0,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [87]:
from mlxtend.frequent_patterns import fpgrowth, fpmax

# mined_comb = fpgrowth(pd.DataFrame(sim_query_list_binary, columns=table_2_search_cols), min_support=0.5, use_colnames=True)
mined_comb = fpmax(pd.DataFrame(sim_query_list_binary, columns=table_2_search_cols), min_support=0.5, use_colnames=True)


In [88]:
for rowid, row in mined_comb.sort_values("support", ascending=False).iterrows():
    print(list(row.itemsets))

['employee: *']
['employee: city', 'employee: name']
